In [1]:
# !pip install dice-ml

In [2]:
from util_comp import *

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import dice_ml


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from time import time

# Makes sure we see all columns
pd.set_option('display.max_columns', None)
# ignore warnings
warnings.filterwarnings("ignore")

In [3]:
data = main(retained=True)

Only retained columns are used
Shape of data: (44535, 72)
Elapsed time to compute age categorisation: 0 minutes and 0 seconds
Elapsed time to compute load column names: 0 minutes and 0 seconds
Elapsed time to compute clean data: 0 minutes and 0 seconds
Elapsed time to compute categorisation of outcome column: 0 minutes and 3 seconds
Dropped rows with missing values
Elapsed time to compute change values in catagorical columns: 1 minutes and 20 seconds
Elapsed time to compute Full process: 1 minutes and 24 seconds


In [4]:
data.describe()

,Gender,Age,Work type,sector,size of business,No of 3 to 6 years old children,No of 7 to 12 years old children,No of 13 to 17 years old children,No of 18 years and over children,dependents,Full Time/Part Time,shift work with alternating hours,Place of work,standing or in an awkward posture,carry or move heavy loads,perform repetitive gestures,work on screen,cold/ hot env,work in noisy env,breathe toxic products or dust,handle toxic or dangerous products,risk of serious fall,machinery with exposure to injury,learn new things,work quickly or very intensely,long periods of concentration,physically tiring,mentally tiring,satisfaction with work,make decisions,difficulty managing priorities,recognized by my hierarchy,develop professional skills,good understanding where I work,well-suited workstation,seek to improve the way I work,well-being of its employees,Security is a priority,count on supervisor,recommend the company,decreased alertness,hurtful or aggressive remarks,insulted or attacked by the public or customers,clarity of your role in the company,access to professional training,tools or workstation,prospects for development,mastery of position,difficulty reconciling your work with your other personal commitments,transportation,personal concerns,can talk to loved ones,state of health in general,long-term illness,Disabilities,Headaches,"neck, shoulder, arm pain",back pain,"leg, foot, knee pain",other parts of the body pain,midday meal,drinks (At least once a week),smoker,"sleeping pills, anxiolytics or anti-depressants",Sleep disorders,lack of tone,outcome
count,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000,23239.000000
mean,0.458798,2.031585,1.544860,4.121520,1.796119,0.175094,0.241405,0.192521,0.204570,1.752141,0.123413,0.772710,2.382504,1.616292,1.882138,1.374500,0.795430,1.786652,1.406515,0.710616,0.803520,0.807393,0.761479,1.165326,1.136366,1.007143,1.573088,1.118206,1.085331,1.108094,1.711649,1.426869,1.289169,1.021559,1.176212,0.931107,1.519342,1.157451,1.244029,1.331469,1.920220,2.298808,2.484358,0.256121,0.483842,0.252033,0.606351,0.199449,1.818021,0.419381,1.806059,0.942812,1.244847,0.795602,0.933775,1.065752,1.065020,1.260510,0.831964,0.586084,1.984466,0.953569,0.759241,0.364043,1.136280,1.087052,0.675502
std,0.498310,1.616583,1.176684,3.035059,1.203711,0.444073,0.560395,0.484223,0.556069,0.602100,0.328918,0.419091,2.199853,1.199583,1.090693,1.145228,1.116339,1.131097,1.075245,0.453486,0.397344,0.394356,0.426189,0.840975,0.814355,0.786232,1.004543,0.850426,0.763038,0.861691,0.841076,0.891368,0.852914,0.777131,0.741809,0.668108,0.836980,0.849732,0.885110,0.875199,0.689516,0.820311,0.767544,0.436499,0.499750,0.434190,0.488569,0.399595,0.856404,0.673305,0.929407,0.812772,0.717852,0.403270,0.248680,0.671475,0.836458,0.813669,0.821510,0.721521,1.550418,1.081873,0.427554,1.033857,0.848326,0.706007,1.087330
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [5]:
# Function to clean column names
def clean_column_names(df):
    df.columns = [str(col).replace(' ', '_').replace('?', '').replace('-', '_').replace('(', '').replace(')', '') for col in df.columns]
    return df

data = clean_column_names(data)

In [6]:
X = data.drop('outcome', axis=1, inplace=False)
X.columns = [str(i) for i in X.columns]
y = data['outcome']

# Split the data for evaluation
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(18591, 66)
(4648, 66)


In [7]:
data[0:1]

,Gender,Age,Work_type,sector,size_of_business,No_of_3_to_6_years_old_children,No_of_7_to_12_years_old_children,No_of_13_to_17_years_old_children,No_of_18_years_and_over_children,dependents,Full_Time/Part_Time,shift_work_with_alternating_hours,Place_of_work,standing_or_in_an_awkward_posture,carry_or_move_heavy_loads,perform_repetitive_gestures,work_on_screen,cold/_hot_env,work_in_noisy_env,breathe_toxic_products_or_dust,handle_toxic_or_dangerous_products,risk_of_serious_fall,machinery_with_exposure_to_injury,learn_new_things,work_quickly_or_very_intensely,long_periods_of_concentration,physically_tiring,mentally_tiring,satisfaction_with_work,make_decisions,difficulty_managing_priorities,recognized_by_my_hierarchy,develop_professional_skills,good_understanding_where_I_work,well_suited_workstation,seek_to_improve_the_way_I_work,well_being_of_its_employees,Security_is_a_priority,count_on_supervisor,recommend_the_company,decreased_alertness,hurtful_or_aggressive_remarks,insulted_or_attacked_by_the_public_or_customers,clarity_of_your_role_in_the_company,access_to_professional_training,tools_or_workstation,prospects_for_development,mastery_of_position,difficulty_reconciling_your_work_with_your_other_personal_commitments,transportation,personal_concerns,can_talk_to_loved_ones,state_of_health_in_general,long_term_illness,Disabilities,Headaches,"neck,_shoulder,_arm_pain",back_pain,"leg,_foot,_knee_pain",other_parts_of_the_body_pain,midday_meal,drinks_At_least_once_a_week,smoker,"sleeping_pills,_anxiolytics_or_anti_depressants",Sleep_disorders,lack_of_tone,outcome
0,1,1,2,7,0,0.0,0.0,0.0,0.0,2,0,1,0,2,1,0,0,3,1,1,1,1,1,1,1,0,2,0,2,2,0,3,1,2,2,1,3,3,3,3,1,0,1,1,0,0,1,1,2,0,1,1,2,1,1,2,3,3,2,1,1,2,0,0,1,2,2


In [8]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(f"F1 Score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")

F1 Score 0.2933870202783485
Accuracy 0.6839500860585198


In [9]:
# non-catagorical columns
non_cat_cols = [
    'No_of_3_to_6_years_old_children',	
    'No_of_7_to_12_years_old_children',	
    'No_of_13_to_17_years_old_children',	
    'No_of_18_years_and_over_children'	
]

# catagorical columns (everything other than the non_categorical columns)
categorical_cols = [col for col in data.columns if col not in non_cat_cols]

In [10]:
st = time()
data_dice = dice_ml.Data(dataframe=data, 
                         # For perturbation strategy
                         continuous_features=non_cat_cols, 
                         outcome_name='outcome')
# Model
rf_dice = dice_ml.Model(model=rf, 
                        # There exist backends for tf, torch, ...
                        backend="sklearn")
explainer = dice_ml.Dice(data_dice, 
                         rf_dice, 
                         # Random sampling, genetic algorithm, kd-tree,...
                         method="random")
et = time()
print(time_e(st, et, "DICE"))

Elapsed time to compute DICE: 0 minutes and 0 seconds


In [11]:
# Show class 2 in the first 30 instances and generate counterfactuals for it to change to class 0
# class 2 is Long Leave
# class 0 is Very Short Leave

a = 0
for i in y_train:
    a = a + 1
    if a == 30:
        break
    if i == 2:
        print(a)
        input_datapoint = X_train[a-1:a]
        cf = explainer.generate_counterfactuals(input_datapoint, 
                                                total_CFs=5, 
                                                desired_class=0 #,
                                                # features_to_vary=features_to_vary,
                                                # permitted_range=permitted_range,
                                                )
        cf.visualize_as_dataframe(show_only_changes=True)

3


100%|██████████| 1/1 [00:00<00:00,  4.19it/s]

Query instance (original outcome : 2)


,Gender,Age,Work_type,sector,size_of_business,No_of_3_to_6_years_old_children,No_of_7_to_12_years_old_children,No_of_13_to_17_years_old_children,No_of_18_years_and_over_children,dependents,Full_Time/Part_Time,shift_work_with_alternating_hours,Place_of_work,standing_or_in_an_awkward_posture,carry_or_move_heavy_loads,perform_repetitive_gestures,work_on_screen,cold/_hot_env,work_in_noisy_env,breathe_toxic_products_or_dust,handle_toxic_or_dangerous_products,risk_of_serious_fall,machinery_with_exposure_to_injury,learn_new_things,work_quickly_or_very_intensely,long_periods_of_concentration,physically_tiring,mentally_tiring,satisfaction_with_work,make_decisions,difficulty_managing_priorities,recognized_by_my_hierarchy,develop_professional_skills,good_understanding_where_I_work,well_suited_workstation,seek_to_improve_the_way_I_work,well_being_of_its_employees,Security_is_a_priority,count_on_supervisor,recommend_the_company,decreased_alertness,hurtful_or_aggressive_remarks,insulted_or_attacked_by_the_public_or_customers,clarity_of_your_role_in_the_company,access_to_professional_training,tools_or_workstation,prospects_for_development,mastery_of_position,difficulty_reconciling_your_work_with_your_other_personal_commitments,transportation,personal_concerns,can_talk_to_loved_ones,state_of_health_in_general,long_term_illness,Disabilities,Headaches,"neck,_shoulder,_arm_pain",back_pain,"leg,_foot,_knee_pain",other_parts_of_the_body_pain,midday_meal,drinks_At_least_once_a_week,smoker,"sleeping_pills,_anxiolytics_or_anti_depressants",Sleep_disorders,lack_of_tone,outcome
0,1,1,3,0,0,1.0,0.0,1.0,0.0,2,0,1,3,0,1,0,2,1,0,0,1,1,0,0,2,2,0,2,2,1,2,1,0,2,1,0,1,1,1,2,1,0,3,0,0,0,0,0,1,0,1,2,2,0,1,3,3,1,0,2,2,2,0,0,1,3,2



Diverse Counterfactual set (new outcome: 0)


...

100%|██████████| 1/1 [00:00<00:00,  5.80it/s]


Query instance (original outcome : 2)


,Gender,Age,Work_type,sector,size_of_business,No_of_3_to_6_years_old_children,No_of_7_to_12_years_old_children,No_of_13_to_17_years_old_children,No_of_18_years_and_over_children,dependents,Full_Time/Part_Time,shift_work_with_alternating_hours,Place_of_work,standing_or_in_an_awkward_posture,carry_or_move_heavy_loads,perform_repetitive_gestures,work_on_screen,cold/_hot_env,work_in_noisy_env,breathe_toxic_products_or_dust,handle_toxic_or_dangerous_products,risk_of_serious_fall,machinery_with_exposure_to_injury,learn_new_things,work_quickly_or_very_intensely,long_periods_of_concentration,physically_tiring,mentally_tiring,satisfaction_with_work,make_decisions,difficulty_managing_priorities,recognized_by_my_hierarchy,develop_professional_skills,good_understanding_where_I_work,well_suited_workstation,seek_to_improve_the_way_I_work,well_being_of_its_employees,Security_is_a_priority,count_on_supervisor,recommend_the_company,decreased_alertness,hurtful_or_aggressive_remarks,insulted_or_attacked_by_the_public_or_customers,clarity_of_your_role_in_the_company,access_to_professional_training,tools_or_workstation,prospects_for_development,mastery_of_position,difficulty_reconciling_your_work_with_your_other_personal_commitments,transportation,personal_concerns,can_talk_to_loved_ones,state_of_health_in_general,long_term_illness,Disabilities,Headaches,"neck,_shoulder,_arm_pain",back_pain,"leg,_foot,_knee_pain",other_parts_of_the_body_pain,midday_meal,drinks_At_least_once_a_week,smoker,"sleeping_pills,_anxiolytics_or_anti_depressants",Sleep_disorders,lack_of_tone,outcome
0,1,3,3,0,1,0.0,0.0,1.0,0.0,2,0,0,3,1,2,1,3,2,1,0,1,1,0,2,1,1,0,1,1,3,2,3,3,1,2,1,2,1,2,1,1,2,3,1,0,1,1,0,2,0,2,0,1,1,1,2,2,2,2,1,4,2,0,0,1,2,2



Diverse Counterfactual set (new outcome: 0)


,Gender,Age,Work_type,sector,size_of_business,No_of_3_to_6_years_old_children,No_of_7_to_12_years_old_children,No_of_13_to_17_years_old_children,No_of_18_years_and_over_children,dependents,Full_Time/Part_Time,shift_work_with_alternating_hours,Place_of_work,standing_or_in_an_awkward_posture,carry_or_move_heavy_loads,perform_repetitive_gestures,work_on_screen,cold/_hot_env,work_in_noisy_env,breathe_toxic_products_or_dust,handle_toxic_or_dangerous_products,risk_of_serious_fall,machinery_with_exposure_to_injury,learn_new_things,work_quickly_or_very_intensely,long_periods_of_concentration,physically_tiring,mentally_tiring,satisfaction_with_work,make_decisions,difficulty_managing_priorities,recognized_by_my_hierarchy,develop_professional_skills,good_understanding_where_I_work,well_suited_workstation,seek_to_improve_the_way_I_work,well_being_of_its_employees,Security_is_a_priority,count_on_supervisor,recommend_the_company,decreased_alertness,hurtful_or_aggressive_remarks,insulted_or_attacked_by_the_public_or_customers,clarity_of_your_role_in_the_company,access_to_professional_training,tools_or_workstation,prospects_for_development,mastery_of_position,difficulty_reconciling_your_work_with_your_other_personal_commitments,transportation,personal_concerns,can_talk_to_loved_ones,state_of_health_in_general,long_term_illness,Disabilities,Headaches,"neck,_shoulder,_arm_pain",back_pain,"leg,_foot,_knee_pain",other_parts_of_the_body_pain,midday_meal,drinks_At_least_once_a_week,smoker,"sleeping_pills,_anxiolytics_or_anti_depressants",Sleep_disorders,lack_of_tone,outcome
0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3.0,-,-,-,-,-,1.0,-,-,-,-,-,-,-,0.0
1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0
2,-,-,-,5.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0
3,-,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0
4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,-,-,-,-,-,-,-,-,-,-,-,-,0.0


In [21]:
query_instance = X_test[0:1]
imp = explainer.local_feature_importance(query_instance, total_CFs=10, desired_class=0)
for i in imp.local_importance:
    for j in i:
        print(j, i[j])

100%|██████████| 1/1 [00:00<00:00,  3.71it/s]

recognized_by_my_hierarchy 0.2
decreased_alertness 0.2
No_of_18_years_and_over_children 0.2
sector 0.1
dependents 0.1
work_in_noisy_env 0.1
seek_to_improve_the_way_I_work 0.1
mastery_of_position 0.1
No_of_3_to_6_years_old_children 0.1
No_of_13_to_17_years_old_children 0.1
Gender 0.0
Age 0.0
Work_type 0.0
size_of_business 0.0
Full_Time/Part_Time 0.0
shift_work_with_alternating_hours 0.0
Place_of_work 0.0
standing_or_in_an_awkward_posture 0.0
carry_or_move_heavy_loads 0.0
perform_repetitive_gestures 0.0
work_on_screen 0.0
cold/_hot_env 0.0
breathe_toxic_products_or_dust 0.0
handle_toxic_or_dangerous_products 0.0
risk_of_serious_fall 0.0
machinery_with_exposure_to_injury 0.0
learn_new_things 0.0
work_quickly_or_very_intensely 0.0
long_periods_of_concentration 0.0
physically_tiring 0.0
mentally_tiring 0.0
satisfaction_with_work 0.0
make_decisions 0.0
difficulty_managing_priorities 0.0
develop_professional_skills 0.0
good_understanding_where_I_work 0.0
well_suited_workstation 0.0
well_being

In [22]:
query_instances = X_test[0:20]
imp = explainer.global_feature_importance(query_instances, total_CFs=10, desired_class=0)
for i in imp.local_importance:
    for j in i:
        print(j, i[j])

100%|██████████| 20/20 [00:03<00:00,  5.94it/s]


No_of_3_to_6_years_old_children 0.2
learn_new_things 0.1
mentally_tiring 0.1
difficulty_managing_priorities 0.1
count_on_supervisor 0.1
clarity_of_your_role_in_the_company 0.1
neck,_shoulder,_arm_pain 0.1
back_pain 0.1
midday_meal 0.1
No_of_7_to_12_years_old_children 0.1
No_of_13_to_17_years_old_children 0.1
Gender 0.0
Age 0.0
Work_type 0.0
sector 0.0
size_of_business 0.0
dependents 0.0
Full_Time/Part_Time 0.0
shift_work_with_alternating_hours 0.0
Place_of_work 0.0
standing_or_in_an_awkward_posture 0.0
carry_or_move_heavy_loads 0.0
perform_repetitive_gestures 0.0
work_on_screen 0.0
cold/_hot_env 0.0
work_in_noisy_env 0.0
breathe_toxic_products_or_dust 0.0
handle_toxic_or_dangerous_products 0.0
risk_of_serious_fall 0.0
machinery_with_exposure_to_injury 0.0
work_quickly_or_very_intensely 0.0
long_periods_of_concentration 0.0
physically_tiring 0.0
satisfaction_with_work 0.0
make_decisions 0.0
recognized_by_my_hierarchy 0.0
develop_professional_skills 0.0
good_understanding_where_I_work 0.